# xView

## Dataset
[Armory Version](#TODO)

In [ ]:
%%bash
# Execute this cell if you are running this notebook in a remote
# jupyter environment. This process may take some time.

IN_COLAB=`python -c "import google.colab"`

if [[ $IN_COLAB -lt 1 ]]; then
  echo "Running setup for Google Colab..."
  git clone https://github.com/twosixlabs/armory.git /armory-repo
  pushd /armory-repo
    pip install --upgrade pip
    pip install ".[pytorch]"
  popd
fi


In [ ]:
# Handle numpy runtime error in Google Colab:
#  https://numpy.org/devdocs/user/troubleshooting-importerror.html#c-api-incompatibility
!pip install numpy==1.23.0 --force
!echo "Please restart the runtime to continue (ctrl + m)..."

### Loading the Scenario
To load the scenario, we use Armory's `get()` function in `armory.scenarios.main`. The filepath used below is an example, and you'll receive the specific path logged to console after invoking `armory run`.

Calling `load()` will load the pieces (model, dataset, attack, etc.) necessary to run the scenario.

In [3]:
# When launched interactively, using --interactive, the scenario config will
# be copied to a time-stamped sub-directory of `/armory/tmp/`.
# Example:
#    >>> # To run, inside of a notebook:
#    >>>     from armory.scenarios.main import get as get_scenario
#    >>>     scenario = get_scenario("/armory/tmp/2022-11-09T130708.219154/interactive-config.json").load()
#    >>>     scenario.evaluate()

import json
from pathlib import Path
from pprint import pprint

from armory.scenarios.main import get as get_scenario

scenario_config = "/armory-repo/scenario_configs/eval1-4/xview/xview_frcnn_sweep_patch_size.json"

# json_config = json.load(Path(scenario_config).read_text())
# pprint(json_config)

scenario = get_scenario(scenario_config).load()

2023-06-12 20:35:37 2m19s ERROR    armory.scenarios.main:_scenario_setup:42 eval_id not in config. Inserting current timestamp 1686602137.4770539
2023-06-12 20:35:37 2m19s INFO     armory.scenarios.main:_scenario_setup:50 armory outputs and logs will be written to /armory/outputs/1686602137.4770539
2023-06-12 20:35:37 2m19s INFO     armory.data.utils:maybe_download_weights_from_s3:47 Using available xview_model_state_dict_epoch_99_loss_0p67 in Armory `saved_model_dir`


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


2023-06-12 20:35:38 2m20s INFO     armory.scenarios.scenario:load_dataset:254 Loading test dataset xview...


# Evaluation

In [ ]:
scenario.evaluate()

2023-06-12 20:37:17 3m58s INFO     armory.data.utils:maybe_download_weights_from_s3:47 Using available xview_model_state_dict_epoch_99_loss_0p67 in Armory `saved_model_dir`
2023-06-12 20:37:18 3m59s INFO     armory.scenarios.scenario:load_dataset:254 Loading test dataset xview...
2023-06-12 20:37:18 3m59s INFO     armory.scenarios.scenario:evaluate_all:332 Running inference on benign and adversarial examples


Evaluation:   0%|          | 0/19277 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

2023-06-12 20:37:47 4m28s INFO     armory.art_experimental.attacks.sweep:generate:121 Failure with kwargs {} and generate_kwargs {'patch_height': 60, 'patch_width': 60}


PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

...

2023-06-12 20:39:34 6m15s INFO     armory.art_experimental.attacks.sweep:generate:121 Failure with kwargs {} and generate_kwargs {'patch_height': 110, 'patch_width': 110}
2023-06-12 20:39:34 6m15s INFO     armory.art_experimental.attacks.sweep:generate:126 Sweep attack concluded. Returning original x since attack failed at all sweep points.


Evaluation:   0%|          | 2/19277 [02:16<360:15:45, 67.29s/it]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

2023-06-12 20:39:56 6m37s INFO     armory.art_experimental.attacks.sweep:generate:121 Failure with kwargs {} and generate_kwargs {'patch_height': 60, 'patch_width': 60}


PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

# TODO


In [ ]:
from tqdm import tqdm


# Update Attack params
# scenario.config["attack"]["kwargs"]["max_iter"] = 500
scenario.load_attack()


for _ in tqdm(range(len(scenario.test_dataset)), desc="Evaluation"):
  scenario.next()
  if not scenario.skip_benign:
    scenario.run_benign()
    try:
      scenario.sample_exporter.export(scenario.x[0], f"benign_batch_{scenario.i}", with_boxes=False)
      if scenario.y[0] != None and scenario.y_pred[0] != None:
        scenario.sample_exporter.export(
          scenario.x[0],
          f"benign_batch_{scenario.i}_bbox",
          y=scenario.y[0],
          y_pred=scenario.y_pred[0],
          with_boxes=True
        )
    except:
      pass
  if not scenario.skip_attack:
    scenario.run_attack()
    try:
      scenario.sample_exporter.export( scenario.x_adv[0], f"adversarial_batch_{scenario.i}", with_boxes=False)
      if scenario.y[0] != None or scenario.y_pred_adv[0] != None:
        scenario.sample_exporter.export(
          scenario.x_adv[0],
          f"adversarial_batch_{scenario.i}_bbox",
          y=scenario.y[0],
          y_pred=scenario.y_pred_adv[0],
          with_boxes=True
        )
    except:
      pass


scenario.hub.set_context(stage="finished")
scenario.finalize_results()

In [ ]:
results_dict = scenario.prepare_results()

print(results_dict["results"])

scenario.save()

In [ ]:
from pathlib import Path

import torch
import torchvision
import torchvision.transforms as T
from art.estimators.object_detection import PyTorchFasterRCNN
from armory.baseline_models.pytorch.carla_single_modality_object_detection_frcnn import get_art_model


class Demo:
  def __init__(self, device, images, checkpoint):
    self.device = device
    self.images = images
    self.model = self.load_model(checkpoint).model

  def load_model(self, checkpoint):
    return get_art_model(
        model_kwargs = {"num_classes": 4},
        wrapper_kwargs = {},
        weights_path = "/armory/saved_models/carla_rgb_weights_eval5.pt",
        )

  @torch.no_grad()
  def detect(self, image):
    transform = T.Compose([T.Resize((960, 1280)), T.ToTensor()])
    image_tensor = transform(image).unsqueeze(0).to(self.device)
    print(image_tensor.shape)

    return self.model(image_tensor)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
output_directory = Path(f"/armory/outputs/{scenario.config['eval_id']}")
output_dataset = Path(f"/datasets/carla_obj_det_dev/2.0/")
output_images = list(Path(output_directory / "saved_samples").glob("*.png"))
output_checkpoint = Path("/armory/saved_models/carla_rgb_weights_eval5.pt")

demo = Demo(device, output_images, output_checkpoint)

In [ ]:
import PIL
import IPython.display as cell


target = PIL.Image.open(demo.images[1])

cell.display(target)
demo.detect(target)

In [ ]:
import tarfile
import os

with tarfile.open("scenario_output.tar.gz", "w:gz") as tar:
    tar.add("/armory", arcname=os.path.basename("/armory"))

In [ ]:
from google.colab import drive

drive.mount(mountpoint="/gdrive", force_remount=True)

import shutil

source_path = "/armory"
destination_path = "/gdrive/armory"

shutil.copyfile(source_path, destination_path)